In [1]:
import pandas as pd
from pathlib import Path
from basketball_reference_scraper.seasons import get_schedule, get_standings
from basketball_reference_scraper.box_scores import get_box_scores
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


In [2]:
data_path = Path('Data')
assert data_path.exists()

In [3]:
with open(data_path/'team_params.txt') as f:
    teams = f.readlines()
team_dict={}
for team in teams:
    splits = team.split(":")
    team_dict[splits[0].strip().title()] = splits[1].strip()
team_dict

{'Atlanta Hawks': 'ATL',
 'St. Louis Hawks': 'SLH',
 'Milwaukee Hawks': 'MIL',
 'Tri-Cities Blackhawks': 'TCB',
 'Boston Celtics': 'BOS',
 'Brooklyn Nets': 'BRK',
 'New Jersey Nets': 'NJN',
 'Chicago Bulls': 'CHI',
 'Charlotte Hornets (1988-2004)': 'CHH',
 'Charlotte Hornets (2014-Present)': 'CHO',
 'Charlotte Bobcats': 'CHA',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Fort Wayne Pistons': 'FWP',
 'Golden State Warriors': 'GSW',
 'San Francisco Warriors': 'SFW',
 'Philadelphia Warriors': 'PHI',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'San Diego Clippers': 'SDC',
 'Buffalo Braves': 'BUF',
 'Los Angeles Lakers': 'LAL',
 'Minneapolis Lakers': 'MIN',
 'Memphis Grizzlies': 'MEM',
 'Vancouver Grizzlies': 'VAN',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New Orleans/Oklahoma City Hornets': 'NOK',
 'New Or

# Get Schedule

In [5]:
all_schedules = []
for year in range(2014,2022):
    print(year)
    year_schedule = get_schedule(year, playoffs=False)
    all_schedules.append(year_schedule)

2014
2015
2016
2017
2018
2019
2020
2021


In [6]:
schedule_20142021 = pd.concat(all_schedules)
len(schedule_20142021)

8980

In [7]:
playedgames_20142021 = schedule_20142021.dropna(how='any')
playedgames_20142021 = playedgames_20142021.replace({"Philadelphia 76ers":"Philadelphia 76Ers"})
playedgames_20142021 = playedgames_20142021.replace(team_dict)
playedgames_20142021 = playedgames_20142021.replace({'Charlotte Hornets':'CHO'})
len(playedgames_20142021)

8744

In [8]:
playedgames_20142021.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS
0,2013-10-29,ORL,87,IND,97
1,2013-10-29,CHI,95,MIA,107
2,2013-10-29,LAC,103,LAL,116
3,2013-10-30,BRK,94,CLE,98
4,2013-10-30,BOS,87,TOR,93


In [9]:
# check if team names are replaced with abbreviation
for visitor in playedgames_20142021['VISITOR'].unique():
    assert len(visitor)==3, f"{visitor}'s team name needs fix"
for home in playedgames_20142021['HOME'].unique():
    assert len(visitor)==3, f"{home}'s team name needs fix"

# Build URLs

In [35]:
box_scores_urls=[]
gameids = []
for key,row in playedgames_20142021.iterrows():
    nums_to_join=[str(num) for num in [row['DATE'].year,f"{row['DATE'].month:02d}",f"{row['DATE'].day:02d}",0,row['HOME']]]
    url = "https://www.basketball-reference.com/boxscores/"+''.join(nums_to_join)+".html"
    gameids.append(''.join(nums_to_join))
    box_scores_urls.append(url)

In [36]:
box_scores_urls[:3]

['https://www.basketball-reference.com/boxscores/201310290IND.html',
 'https://www.basketball-reference.com/boxscores/201310290MIA.html',
 'https://www.basketball-reference.com/boxscores/201310290LAL.html']

In [12]:
len(box_scores_urls)

8744

In [37]:
playedgames_20142021['boxscores_url'] = box_scores_urls
playedgames_20142021['game_id'] = gameids
playedgames_20142021.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR


# Scrape four factors

In [40]:
tables = []
bad_games =[]
for key,row in tqdm(playedgames_20142021.iterrows()):
    try:
        response = requests.get(row['boxscores_url'])
        html = response.content
        html = html.decode()
        stat_html = html.replace('<!--', "")
        stat_html = stat_html.replace('-->', "")
        soup = BeautifulSoup(stat_html, 'html.parser')
        table = pd.read_html(str(soup.find_all('table',attrs={"id":"four_factors"})[0]))[0]
        table = table.droplevel(0,axis=1)
        table = table.rename({'Unnamed: 0_level_1':'Team'},axis=1)
        table['game_id'] = row['game_id']
        tables.append(table)
        if key%100==0: print(key)
    except:
        bad_games.append(row['game_id'])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300



In [41]:
temp_four_factors = pd.concat(tables)
temp_four_factors.head()

,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,ORL,95.0,0.435,14.9,27.7,0.065,91.6,201310290IND
1,IND,95.0,0.528,19.0,27.8,0.310,102.1,201310290IND
0,CHI,97.1,0.464,16.2,23.9,0.217,97.9,201310290MIA
1,MIA,97.1,0.590,17.5,14.3,0.306,110.2,201310290MIA
0,LAC,100.4,0.542,14.7,22.7,0.157,102.6,201310290LAL


In [42]:
temp_four_factors.to_pickle("Data/temp_four_factors.pkl")

In [6]:
temp_four_factors = pd.read_pickle("Data/temp_four_factors.pkl")

In [7]:
temp_four_factors

,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,ORL,95.0,0.435,14.9,27.7,0.065,91.6,201310290IND
1,IND,95.0,0.528,19.0,27.8,0.310,102.1,201310290IND
0,CHI,97.1,0.464,16.2,23.9,0.217,97.9,201310290MIA
1,MIA,97.1,0.590,17.5,14.3,0.306,110.2,201310290MIA
0,LAC,100.4,0.542,14.7,22.7,0.157,102.6,201310290LAL
...,...,...,...,...,...,...,...,...
1,WAS,96.6,0.651,12.6,16.7,0.157,125.3,202102020WAS
0,BOS,96.4,0.542,13.5,28.6,0.238,115.1,202102020GSW
1,GSW,96.4,0.548,10.6,13.3,0.193,111.0,202102020GSW
0,DET,99.6,0.468,12.8,25.0,0.194,105.4,202102020UTA


In [47]:
# handle the bad games
print(len(bad_games))
print(bad_games)

1
['201711110GSW']


# Join Four Factors with schedule to construct training data

In [52]:
temp_four_factors.head()

,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,ORL,95.0,0.435,14.9,27.7,0.065,91.6,201310290IND
1,IND,95.0,0.528,19.0,27.8,0.310,102.1,201310290IND
0,CHI,97.1,0.464,16.2,23.9,0.217,97.9,201310290MIA
1,MIA,97.1,0.590,17.5,14.3,0.306,110.2,201310290MIA
0,LAC,100.4,0.542,14.7,22.7,0.157,102.6,201310290LAL


In [53]:
playedgames_20142021.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR


In [55]:
# join on home team
home_joined=pd.merge(playedgames_20142021,temp_four_factors,left_on=['game_id','HOME'],right_on=['game_id','Team'])
home_joined

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND,IND,95.0,0.528,19.0,27.8,0.310,102.1
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA,MIA,97.1,0.590,17.5,14.3,0.306,110.2
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL,LAL,100.4,0.527,15.3,37.5,0.194,115.6
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE,CLE,92.3,0.446,10.0,36.4,0.274,106.2
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR,TOR,91.5,0.471,15.0,42.2,0.140,101.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738,2021-02-02,LAC,120,BRK,124,https://www.basketball-reference.com/boxscores...,202102020BRK,BRK,99.9,0.652,15.0,17.6,0.266,124.1
8739,2021-02-02,MEM,116,IND,134,https://www.basketball-reference.com/boxscores...,202102020IND,IND,104.2,0.695,16.0,25.7,0.244,128.6
8740,2021-02-02,POR,132,WAS,121,https://www.basketball-reference.com/boxscores...,202102020WAS,WAS,96.6,0.651,12.6,16.7,0.157,125.3
8741,2021-02-02,BOS,111,GSW,107,https://www.basketball-reference.com/boxscores...,202102020GSW,GSW,96.4,0.548,10.6,13.3,0.193,111.0


In [56]:
joined=pd.merge(home_joined,temp_four_factors,left_on=['game_id','VISITOR'],right_on=['game_id','Team'],suffixes=('_home','_visitor'))
joined.columns

Index(['DATE', 'VISITOR', 'VISITOR_PTS', 'HOME', 'HOME_PTS', 'boxscores_url',
       'game_id', 'Team_home', 'Pace_home', 'eFG%_home', 'TOV%_home',
       'ORB%_home', 'FT/FGA_home', 'ORtg_home', 'Team_visitor', 'Pace_visitor',
       'eFG%_visitor', 'TOV%_visitor', 'ORB%_visitor', 'FT/FGA_visitor',
       'ORtg_visitor'],
      dtype='object')

In [57]:
joined

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id,Team_home,Pace_home,eFG%_home,...,ORB%_home,FT/FGA_home,ORtg_home,Team_visitor,Pace_visitor,eFG%_visitor,TOV%_visitor,ORB%_visitor,FT/FGA_visitor,ORtg_visitor
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND,IND,95.0,0.528,...,27.8,0.310,102.1,ORL,95.0,0.435,14.9,27.7,0.065,91.6
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA,MIA,97.1,0.590,...,14.3,0.306,110.2,CHI,97.1,0.464,16.2,23.9,0.217,97.9
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL,LAL,100.4,0.527,...,37.5,0.194,115.6,LAC,100.4,0.542,14.7,22.7,0.157,102.6
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE,CLE,92.3,0.446,...,36.4,0.274,106.2,BRK,92.3,0.457,13.9,22.0,0.232,101.8
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR,TOR,91.5,0.471,...,42.2,0.140,101.6,BOS,91.5,0.508,21.8,19.4,0.303,95.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738,2021-02-02,LAC,120,BRK,124,https://www.basketball-reference.com/boxscores...,202102020BRK,BRK,99.9,0.652,...,17.6,0.266,124.1,LAC,99.9,0.530,7.0,22.4,0.152,120.1
8739,2021-02-02,MEM,116,IND,134,https://www.basketball-reference.com/boxscores...,202102020IND,IND,104.2,0.695,...,25.7,0.244,128.6,MEM,104.2,0.500,13.4,23.3,0.247,111.3
8740,2021-02-02,POR,132,WAS,121,https://www.basketball-reference.com/boxscores...,202102020WAS,WAS,96.6,0.651,...,16.7,0.157,125.3,POR,96.6,0.574,7.9,29.8,0.242,136.7
8741,2021-02-02,BOS,111,GSW,107,https://www.basketball-reference.com/boxscores...,202102020GSW,GSW,96.4,0.548,...,13.3,0.193,111.0,BOS,96.4,0.542,13.5,28.6,0.238,115.1


In [58]:
joined.to_pickle("Data/full_data_2014to2021.pkl")

In [4]:
joined = pd.read_pickle("Data/full_data_2014to2021.pkl")

In [5]:
joined.head()

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,boxscores_url,game_id,Team_home,Pace_home,eFG%_home,...,ORB%_home,FT/FGA_home,ORtg_home,Team_visitor,Pace_visitor,eFG%_visitor,TOV%_visitor,ORB%_visitor,FT/FGA_visitor,ORtg_visitor
0,2013-10-29,ORL,87,IND,97,https://www.basketball-reference.com/boxscores...,201310290IND,IND,95.0,0.528,...,27.8,0.310,102.1,ORL,95.0,0.435,14.9,27.7,0.065,91.6
1,2013-10-29,CHI,95,MIA,107,https://www.basketball-reference.com/boxscores...,201310290MIA,MIA,97.1,0.590,...,14.3,0.306,110.2,CHI,97.1,0.464,16.2,23.9,0.217,97.9
2,2013-10-29,LAC,103,LAL,116,https://www.basketball-reference.com/boxscores...,201310290LAL,LAL,100.4,0.527,...,37.5,0.194,115.6,LAC,100.4,0.542,14.7,22.7,0.157,102.6
3,2013-10-30,BRK,94,CLE,98,https://www.basketball-reference.com/boxscores...,201310300CLE,CLE,92.3,0.446,...,36.4,0.274,106.2,BRK,92.3,0.457,13.9,22.0,0.232,101.8
4,2013-10-30,BOS,87,TOR,93,https://www.basketball-reference.com/boxscores...,201310300TOR,TOR,91.5,0.471,...,42.2,0.140,101.6,BOS,91.5,0.508,21.8,19.4,0.303,95.1


# Add Defensive Rating

## Scrape Box Scores

In [101]:
joined = pd.read_pickle("Data/full_data_2014to2021.pkl")

# from multiprocessing.dummy import Pool as ThreadPool
params = list(zip(list(joined['game_id']),list(joined['DATE']),list(joined['HOME']),list(joined['VISITOR']),['GAME']*len(joined),['ADVANCED']*len(joined)))
box_scores={}
bad_games =[]
for param in tqdm(params,total = len(params)):
    try:
        box_scores[param[0]]=get_box_scores(*param[1:])
    except Exception as exc:
        print(exc,param)
        print('-'*50)
        bad_games.append(param)

HTTPSConnectionPool(host='widgets.sports-reference.com', port=443): Max retries exceeded with url: /wg.fcgi?css=1&site=bbr&url=%2Fboxscores%2F201612210PHO.html&div=div_box-PHO-game-advanced (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3a3c87a4d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')) ('201612210PHO', Timestamp('2016-12-21 00:00:00'), 'PHO', 'HOU', 'GAME', 'ADVANCED')
--------------------------------------------------



In [102]:
bad_games

[('201612210PHO',
  Timestamp('2016-12-21 00:00:00'),
  'PHO',
  'HOU',
  'GAME',
  'ADVANCED')]

In [109]:
[1,2,3].pop()

3

In [107]:
more_bad_games=[]
for param in tqdm(bad_games,total = len(bad_games)):
    try:
        box_scores[param[0]]=get_box_scores(*param[1:])
    except Exception as exc:
        print(exc,param)
        print('-'*50)
        more_bad_games.append(param)

In [111]:
bad_games.pop()

('201612210PHO',
 Timestamp('2016-12-21 00:00:00'),
 'PHO',
 'HOU',
 'GAME',
 'ADVANCED')

In [112]:
assert len(bad_games)==0
assert len(more_bad_games)==0

In [113]:
len(joined)

8743

In [114]:
len(box_scores)

8743

In [115]:
# save the data
import pickle
fname = "Data/box_scores_2014to2021.p"
pickle.dump(box_scores,open(fname, "wb"))

In [63]:
# get only team level data
box_scores = pickle.load(open(fname, "rb" ))

In [64]:
len(box_scores)

8743

In [94]:
updated_box_score = []
for game_id, team_box_score in box_scores.items():
    for team, box_score in team_box_score.items():
        print(team)
        box_score['Team']=team
        box_score['game_id']=game_id
        updated_box_score.append(box_score)

IND
              PLAYER            MP           TS%          eFG%          3PAr  \
0        Paul George         36:05          .629          .594          .375   
1   Lance Stephenson         35:38          .738          .750          .250   
2         David West         30:43          .492          .364          .000   
3        George Hill         29:04          .359          .250          .250   
4        Roy Hibbert         26:54          .515          .500          .000   
5         Luis Scola         20:43          .571          .571          .000   
6      D.J. Augustin         19:54          .425          .500          .800   
7        Ian Mahinmi         16:42          .549          .000          .000   
8    Orlando Johnson         15:12          .922          .875          .500   
9       Solomon Hill          8:07          .000          .000          .000   
10     Rasual Butler          0:58           NaN           NaN           NaN   
11    Chris Copeland  Did Not Play  

                   PLAYER             MP            TS%           eFG%  \
0             J.J. Redick          35:57           .523           .500   
1            Caron Butler          33:26           .709           .667   
2         Khris Middleton          19:15           .250           .250   
3           Larry Sanders          12:21           .000           .000   
4          Brandon Knight           1:45            NaN            NaN   
5           Zaza Pachulia          33:53           .623           .600   
6               Gary Neal          31:49           .474           .438   
7            Nate Wolters          29:41           .349           .292   
8          Ersan İlyasova          19:43           .833           .833   
9             John Henson          17:27           .868          1.000   
10  Giannis Antetokounmpo           4:43           .568            NaN   
11     Miroslav Raduljica   Did Not Play   Did Not Play   Did Not Play   
12           Luke Ridnour  Did Not Dre